# Expected Goal Model using logistic regression

In [1]:
import sys
import os
import pandas as pd
import numpy as np

import math

project_root = os.path.abspath("..")

if project_root not in sys.path:
    sys.path.append(project_root)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import pyodbc
from sqlalchemy import create_engine
from preprocessing.event_data import load_and_process_event_data


In [2]:
df_events = load_and_process_event_data()


In [3]:
df_shots = df_events[df_events['eventName'] == 'Shot'].copy()

In [4]:
df_shots.head()

,matchID,matchPeriod,eventSec,eventName,subEventName,teamID,playerID,playerName,playerPosition,playerStrongFoot,...,awayTeamId,Goal,OwnGoal,CounterAttack,bodyPartShot,bodyPartShotCode,posOrigXMeters,posDestXMeters,posOrigYMeters,posDestYMeters
24,1694390,1H,31.226217,Shot,Shot,4418,25437.0,B. Matuidi,Midfielder,left,...,11944,0,0,0,rightFoot,1.0,95.55,0.0,19.72,0.00
61,1694390,2H,100.604872,Shot,Shot,11944,83753.0,N. Stanciu,Midfielder,right,...,11944,0,0,0,rightFoot,2.0,105.00,21.0,68.00,45.56
85,1694390,2H,130.592908,Shot,Shot,11944,33235.0,B. Stancu,Forward,right,...,11944,0,0,0,rightFoot,2.0,105.00,0.0,68.00,0.00
89,1694390,1H,143.119551,Shot,Shot,11944,83824.0,M. Pintilii,Midfielder,right,...,11944,0,0,0,rightFoot,2.0,74.55,105.0,19.72,68.00
137,1694390,1H,219.576026,Shot,Shot,11944,33235.0,B. Stancu,Forward,right,...,11944,0,0,0,rightFoot,2.0,100.80,105.0,38.76,68.00


In [5]:
total_shots = len(df_shots)
print(f"Tổng số cú sút: {total_shots}")
print(f"Xác suất ghi bàn khi thực hiện cú sút: {df_shots['Goal'].mean()*100:.2f}%")

Tổng số cú sút: 43071
Xác suất ghi bàn khi thực hiện cú sút: 10.42%


In [7]:
df_events[(df_events['subEventName'] == 'Goal kick') & (df_events['playerID'] == 50849)].head(20)

,matchID,matchPeriod,eventSec,eventName,subEventName,teamID,playerID,playerName,playerPosition,playerStrongFoot,...,awayTeamId,Goal,OwnGoal,CounterAttack,bodyPartShot,bodyPartShotCode,posOrigXMeters,posDestXMeters,posOrigYMeters,posDestYMeters
1974,1694391,2H,594.258748,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,61.95,34.0,45.56
2192,1694391,2H,972.681271,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,70.35,34.0,42.16
2193,1694391,1H,988.950778,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,70.35,34.0,45.56
2386,1694391,2H,1352.070483,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,40.95,34.0,18.36
2637,1694391,2H,1821.159704,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,45.15,34.0,45.56
2841,1694391,2H,2197.216352,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,67.20,34.0,50.32
2921,1694391,2H,2359.381294,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,23.10,34.0,63.24
2953,1694391,1H,2404.978511,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,38.85,34.0,18.36
2976,1694391,1H,2456.406848,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,40.95,34.0,19.72
3125,1694391,1H,2745.455884,Free Kick,Goal kick,8731,50849.0,E. Berisha,Goalkeeper,left,...,6697,0,0,0,None,NaN,5.25,44.10,34.0,8.16
